<h1>Capstone Project - The Battle of Neighborhoods</h1>
<h2>Importing libraries
    


In [6]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import urllib.request
import json
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib inline
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Libraries imported.')

SyntaxError: invalid syntax (<ipython-input-6-4a9034599a99>, line 6)

<h1>Downloading data</h1>
The data we want to download is available free on the internet you can find it in the link below
<a href = "https://geo.nyu.edu/catalog/nyu_2451_34572" > link for the file</a>
for our convinence we have downloaded the jason file and kept in the server we can download it


<h2>Transforming data into pandas dataframe</h2>

In [11]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Data downloaded!


In [12]:
neighborhoods_data = newyork_data['features']
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [13]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


<h3> we will use geopy library to get the latitude and longitude values of the NewYork city</h3>

In [14]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of New York City are 40.7127281, -74.0060152.


<h1>create a map of newyork imposing neighborhoods on top

In [15]:
import folium
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

<h1>Foursquare venues

In [16]:
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',  
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [17]:


CLIENT_ID = 'E3NOAHKHP0NZ2IY0YQ1QJR210XYBGRMAJJYKVWTN5Z2CYQKX' # your Foursquare ID
CLIENT_SECRET = 'C52QBLTPO1ZB2WRJH5TN3CCAPT0ODEAXQID3Q3CAF1B2ULVL' # your Foursquare Secret
VERSION = '20180611'
LIMIT = 500
radius=5000

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: E3NOAHKHP0NZ2IY0YQ1QJR210XYBGRMAJJYKVWTN5Z2CYQKX
CLIENT_SECRET:C52QBLTPO1ZB2WRJH5TN3CCAPT0ODEAXQID3Q3CAF1B2ULVL


In [18]:
#https://developer.foursquare.com/docs/resources/categories
#Stadium = 4bf58dd8d48988d184941735
# footballstadium = 4bf58dd8d48988d189941735
#neighborhoods = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
newyork_venues_football = getNearbyVenues(names=neighborhoods['Neighborhood'], latitudes=neighborhoods['Latitude'], longitudes=neighborhoods['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d184941735')
newyork_venues_football

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Fieldston,40.895437,-73.905643,Van Cortlandt Stadium,40.888585,-73.897433,Baseball Stadium
1,Kingsbridge,40.881687,-73.902818,Van Cortlandt Stadium,40.888585,-73.897433,Baseball Stadium
2,Kingsbridge,40.881687,-73.902818,TCR The Club of Riverdale,40.878628,-73.914568,Tennis Stadium
3,Marble Hill,40.876551,-73.910660,TCR The Club of Riverdale,40.878628,-73.914568,Tennis Stadium
4,Marble Hill,40.876551,-73.910660,Rocco B. Commisso Stadium,40.871884,-73.913897,Soccer Stadium
5,Pelham Parkway,40.857413,-73.854756,NY Giant Heaven,40.856223,-73.843575,Football Stadium
6,High Bridge,40.836623,-73.926102,Empire Parking,40.834902,-73.921777,Baseball Stadium
7,High Bridge,40.836623,-73.926102,Joseph Yancey Track And Field,40.828054,-73.928103,Track Stadium
8,High Bridge,40.836623,-73.926102,Yankee Stadium,40.829869,-73.926584,Baseball Stadium
9,High Bridge,40.836623,-73.926102,Yankee Cap,40.829485,-73.927592,Baseball Stadium


In [19]:
newyork_venues_football.shape

(184, 7)

In [20]:
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Neighborhood'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [21]:
map_newyork_football = folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(newyork_venues_football, 'red', map_newyork_football)

map_newyork_football

In [22]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Neighborhood').count()
    
    for n in startDf['Neighborhood']:
        try:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = 0

In [23]:
ny_grouped = newyork_venues_football.groupby('Neighborhood').count()
ny_grouped

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Astoria,1,1,1,1,1,1
Bay Terrace,1,1,1,1,1,1
Boerum Hill,2,2,2,2,2,2
Brookville,1,1,1,1,1,1
Bulls Head,2,2,2,2,2,2
Carnegie Hill,1,1,1,1,1,1
Carroll Gardens,2,2,2,2,2,2
Castle Hill,1,1,1,1,1,1
Chelsea,2,2,2,2,2,2


<h1> analysing each neighbourhood</h1>

In [24]:
# one hot encoding
ny_onehot = pd.get_dummies(newyork_venues_football[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ny_onehot['Neighborhood'] = newyork_venues_football['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [ny_onehot.columns[-1]] + list(ny_onehot.columns[:-1])
ny_onehot = ny_onehot[fixed_columns]

ny_onehot

,Neighborhood,Baseball Field,Baseball Stadium,Basketball Stadium,College Gym,Conference Room,Food Service,Football Stadium,Historic Site,Hockey Arena,Soccer Field,Soccer Stadium,Sports Club,Stadium,Tennis Court,Tennis Stadium,Track Stadium
0,Fieldston,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Kingsbridge,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Kingsbridge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,Marble Hill,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
5,Pelham Parkway,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
6,High Bridge,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,High Bridge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
8,High Bridge,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,High Bridge,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [25]:
ny_grouped = ny_onehot.groupby('Neighborhood').mean().reset_index()
ny_grouped

,Neighborhood,Baseball Field,Baseball Stadium,Basketball Stadium,College Gym,Conference Room,Food Service,Football Stadium,Historic Site,Hockey Arena,Soccer Field,Soccer Stadium,Sports Club,Stadium,Tennis Court,Tennis Stadium,Track Stadium
0,Astoria,0.0,0.000000,0.000000,0.0,0.000000,0.000000,1.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000
1,Bay Terrace,0.0,0.000000,0.000000,0.0,0.000000,0.000000,1.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000
2,Boerum Hill,0.0,0.000000,0.500000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.5,0.000000,0.0,0.000000,0.000000
3,Brookville,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000
4,Bulls Head,0.0,0.500000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.500000,0.0,0.000000,0.000000
5,Carnegie Hill,0.0,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000
6,Carroll Gardens,0.0,1.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000
7,Castle Hill,0.0,0.000000,0.000000,0.0,0.000000,0.000000,1.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000
8,Chelsea,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.500000,0.000000,0.0,0.0,0.0,0.0,0.500000,0.0,0.000000,0.000000
9,Clinton,0.0,0.200000,0.200000,0.0,0.200000,0.000000,0.200000,0.000000,0.0,0.0,0.0,0.0,0.200000,0.0,0.000000,0.000000


In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [27]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = ny_grouped['Neighborhood']

for ind in np.arange(ny_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ny_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Astoria,Football Stadium,Track Stadium,Tennis Stadium,Tennis Court,Stadium,Sports Club,Soccer Stadium,Soccer Field,Hockey Arena,Historic Site
1,Bay Terrace,Football Stadium,Track Stadium,Tennis Stadium,Tennis Court,Stadium,Sports Club,Soccer Stadium,Soccer Field,Hockey Arena,Historic Site
2,Boerum Hill,Sports Club,Basketball Stadium,Track Stadium,Tennis Stadium,Tennis Court,Stadium,Soccer Stadium,Soccer Field,Hockey Arena,Historic Site
3,Brookville,Hockey Arena,Track Stadium,Tennis Stadium,Tennis Court,Stadium,Sports Club,Soccer Stadium,Soccer Field,Historic Site,Football Stadium
4,Bulls Head,Stadium,Baseball Stadium,Track Stadium,Tennis Stadium,Tennis Court,Sports Club,Soccer Stadium,Soccer Field,Hockey Arena,Historic Site
5,Carnegie Hill,College Gym,Track Stadium,Tennis Stadium,Tennis Court,Stadium,Sports Club,Soccer Stadium,Soccer Field,Hockey Arena,Historic Site
6,Carroll Gardens,Baseball Stadium,Track Stadium,Tennis Stadium,Tennis Court,Stadium,Sports Club,Soccer Stadium,Soccer Field,Hockey Arena,Historic Site
7,Castle Hill,Football Stadium,Track Stadium,Tennis Stadium,Tennis Court,Stadium,Sports Club,Soccer Stadium,Soccer Field,Hockey Arena,Historic Site
8,Chelsea,Stadium,Football Stadium,Track Stadium,Tennis Stadium,Tennis Court,Sports Club,Soccer Stadium,Soccer Field,Hockey Arena,Historic Site
9,Clinton,Stadium,Football Stadium,Conference Room,Basketball Stadium,Baseball Stadium,Track Stadium,Tennis Stadium,Tennis Court,Sports Club,Soccer Stadium


<h1>clustering neighborhoods</h1>

In [28]:
# set number of clusters
kclusters = 5
ny_grouped_clustering = ny_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ny_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 2, 4, 1, 4, 1, 0, 4, 4], dtype=int32)

In [48]:
# add clustering labels

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
ny_merged = neighborhoods
ny_merged = ny_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

ny_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bronx,Wakefield,40.894705,-73.847201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Bronx,Co-op City,40.874294,-73.829939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Bronx,Eastchester,40.887556,-73.827806,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Bronx,Fieldston,40.895437,-73.905643,1.0,Baseball Stadium,Track Stadium,Tennis Stadium,Tennis Court,Stadium,Sports Club,Soccer Stadium,Soccer Field,Hockey Arena,Historic Site
4,Bronx,Riverdale,40.890834,-73.912585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:

ny_merged.fillna(0, inplace=True)
ny_merged['Cluster Labels'] = ny_merged['Cluster Labels'].astype(int)

In [53]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ny_merged['Latitude'], ny_merged['Longitude'], ny_merged['Neighborhood'], ny_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [54]:
ny_merged.loc[ny_merged['Cluster Labels'] == 0, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Wakefield,0,0,0,0,0,0,0,0,0,0
1,Co-op City,0,0,0,0,0,0,0,0,0,0
2,Eastchester,0,0,0,0,0,0,0,0,0,0
4,Riverdale,0,0,0,0,0,0,0,0,0,0
7,Woodlawn,0,0,0,0,0,0,0,0,0,0
8,Norwood,0,0,0,0,0,0,0,0,0,0
9,Williamsbridge,0,0,0,0,0,0,0,0,0,0
10,Baychester,0,0,0,0,0,0,0,0,0,0
11,Pelham Parkway,Football Stadium,Track Stadium,Tennis Stadium,Tennis Court,Stadium,Sports Club,Soccer Stadium,Soccer Field,Hockey Arena,Historic Site
12,City Island,0,0,0,0,0,0,0,0,0,0


In [55]:
ny_merged.loc[ny_merged['Cluster Labels'] == 1, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Fieldston,Baseball Stadium,Track Stadium,Tennis Stadium,Tennis Court,Stadium,Sports Club,Soccer Stadium,Soccer Field,Hockey Arena,Historic Site
19,High Bridge,Baseball Stadium,Stadium,Track Stadium,Football Stadium,Tennis Stadium,Tennis Court,Sports Club,Soccer Stadium,Soccer Field,Hockey Arena
43,Concourse,Baseball Stadium,Stadium,Football Stadium,Track Stadium,Tennis Stadium,Tennis Court,Sports Club,Soccer Stadium,Soccer Field,Hockey Arena
65,Cobble Hill,Baseball Stadium,Track Stadium,Tennis Stadium,Tennis Court,Stadium,Sports Club,Soccer Stadium,Soccer Field,Hockey Arena,Historic Site
66,Carroll Gardens,Baseball Stadium,Track Stadium,Tennis Stadium,Tennis Court,Stadium,Sports Club,Soccer Stadium,Soccer Field,Hockey Arena,Historic Site
67,Red Hook,Baseball Stadium,Track Stadium,Tennis Stadium,Tennis Court,Stadium,Sports Club,Soccer Stadium,Soccer Field,Hockey Arena,Historic Site
68,Gowanus,Baseball Stadium,Track Stadium,Tennis Stadium,Tennis Court,Stadium,Sports Club,Soccer Stadium,Soccer Field,Hockey Arena,Historic Site
78,Coney Island,Baseball Stadium,Track Stadium,Tennis Stadium,Tennis Court,Stadium,Sports Club,Soccer Stadium,Soccer Field,Hockey Arena,Historic Site
106,East Harlem,Baseball Stadium,Track Stadium,Tennis Stadium,Tennis Court,Stadium,Sports Club,Soccer Stadium,Soccer Field,Hockey Arena,Historic Site
111,Upper West Side,Football Stadium,Baseball Stadium,Track Stadium,Tennis Stadium,Tennis Court,Stadium,Sports Club,Soccer Stadium,Soccer Field,Hockey Arena


In [56]:
ny_merged.loc[ny_merged['Cluster Labels'] == 2, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
59,Prospect Heights,Sports Club,Basketball Stadium,Track Stadium,Tennis Stadium,Tennis Court,Stadium,Soccer Stadium,Soccer Field,Hockey Arena,Historic Site
69,Fort Greene,Sports Club,Basketball Stadium,Track Stadium,Tennis Stadium,Tennis Court,Stadium,Soccer Stadium,Soccer Field,Hockey Arena,Historic Site
70,Park Slope,Sports Club,Basketball Stadium,Track Stadium,Tennis Stadium,Tennis Court,Stadium,Soccer Stadium,Soccer Field,Hockey Arena,Historic Site
86,Downtown,Sports Club,Basketball Stadium,Track Stadium,Tennis Stadium,Tennis Court,Stadium,Soccer Stadium,Soccer Field,Hockey Arena,Historic Site
87,Boerum Hill,Sports Club,Basketball Stadium,Track Stadium,Tennis Stadium,Tennis Court,Stadium,Soccer Stadium,Soccer Field,Hockey Arena,Historic Site


In [57]:
ny_merged.loc[ny_merged['Cluster Labels'] == 3, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Kingsbridge,Tennis Stadium,Baseball Stadium,Track Stadium,Tennis Court,Stadium,Sports Club,Soccer Stadium,Soccer Field,Hockey Arena,Historic Site
6,Marble Hill,Tennis Stadium,Soccer Stadium,Track Stadium,Tennis Court,Stadium,Sports Club,Soccer Field,Hockey Arena,Historic Site,Football Stadium
35,Spuyten Duyvil,Tennis Stadium,Soccer Stadium,Track Stadium,Tennis Court,Stadium,Sports Club,Soccer Field,Hockey Arena,Historic Site,Football Stadium
115,Murray Hill,Tennis Stadium,Stadium,Track Stadium,Tennis Court,Sports Club,Soccer Stadium,Soccer Field,Hockey Arena,Historic Site,Football Stadium
134,Corona,Tennis Stadium,Track Stadium,Tennis Court,Stadium,Sports Club,Soccer Stadium,Soccer Field,Hockey Arena,Historic Site,Football Stadium
135,Forest Hills,Tennis Stadium,Track Stadium,Tennis Court,Stadium,Sports Club,Soccer Stadium,Soccer Field,Hockey Arena,Historic Site,Football Stadium
180,Murray Hill,Tennis Stadium,Stadium,Track Stadium,Tennis Court,Sports Club,Soccer Stadium,Soccer Field,Hockey Arena,Historic Site,Football Stadium
195,North Corona,Tennis Stadium,Baseball Stadium,Stadium,Historic Site,Food Service,Track Stadium,Tennis Court,Sports Club,Soccer Stadium,Soccer Field
196,Forest Hills Gardens,Tennis Stadium,Track Stadium,Tennis Court,Stadium,Sports Club,Soccer Stadium,Soccer Field,Hockey Arena,Historic Site,Football Stadium
273,Turtle Bay,Tennis Stadium,Track Stadium,Tennis Court,Stadium,Sports Club,Soccer Stadium,Soccer Field,Hockey Arena,Historic Site,Football Stadium


In [58]:
ny_merged.loc[ny_merged['Cluster Labels'] == 4, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,Sunset Park,Stadium,Track Stadium,Tennis Stadium,Tennis Court,Sports Club,Soccer Stadium,Soccer Field,Hockey Arena,Historic Site,Football Stadium
96,North Side,Track Stadium,Tennis Stadium,Tennis Court,Stadium,Sports Club,Soccer Stadium,Soccer Field,Hockey Arena,Historic Site,Football Stadium
97,South Side,Track Stadium,Tennis Stadium,Tennis Court,Stadium,Sports Club,Soccer Stadium,Soccer Field,Hockey Arena,Historic Site,Football Stadium
102,Inwood,Soccer Stadium,Track Stadium,Tennis Stadium,Tennis Court,Stadium,Sports Club,Soccer Field,Hockey Arena,Historic Site,Football Stadium
104,Manhattanville,Tennis Court,Track Stadium,Tennis Stadium,Stadium,Sports Club,Soccer Stadium,Soccer Field,Hockey Arena,Historic Site,Football Stadium
107,Upper East Side,College Gym,Track Stadium,Tennis Stadium,Tennis Court,Stadium,Sports Club,Soccer Stadium,Soccer Field,Hockey Arena,Historic Site
108,Yorkville,College Gym,Track Stadium,Tennis Stadium,Tennis Court,Stadium,Sports Club,Soccer Stadium,Soccer Field,Hockey Arena,Historic Site
113,Clinton,Stadium,Football Stadium,Conference Room,Basketball Stadium,Baseball Stadium,Track Stadium,Tennis Stadium,Tennis Court,Sports Club,Soccer Stadium
114,Midtown,Stadium,Tennis Stadium,Conference Room,Basketball Stadium,Baseball Stadium,Track Stadium,Tennis Court,Sports Club,Soccer Stadium,Soccer Field
116,Chelsea,Stadium,Football Stadium,Track Stadium,Tennis Stadium,Tennis Court,Sports Club,Soccer Stadium,Soccer Field,Hockey Arena,Historic Site
